In [1]:
#https://www.youtube.com/watch?v=vrrD4DnMZMU&t=9s&ab_channel=AbdelrahmanEzz
import glob
import cv2
from tqdm import tqdm
from skimage import color
x = []
y = []
images4 = glob.glob(r"E:\Z-Machine Learning\My Work\DataSets\Face Expression Recognition\My Own Faces Data\Happy\*")
images5 = glob.glob(r"E:\Z-Machine Learning\My Work\DataSets\Face Expression Recognition\My Own Faces Data\Neutral\*")
images6 = glob.glob(r"E:\Z-Machine Learning\My Work\DataSets\Face Expression Recognition\My Own Faces Data\Sad\*")
ind = 0
for i in tqdm(images4):
    img = cv2.imread(i,3)
    gray = color.rgb2gray(img)
    x.append(gray)
    y.append(0)
    ind += 1
    if(ind == 10000):
        break

ind = 0
for i in tqdm(images5):
    img = cv2.imread(i,3)
    gray = color.rgb2gray(img)
    x.append(gray)
    y.append(1)
    ind += 1
    if(ind == 10000):
        break
        
ind = 0
for i in tqdm(images6):
    img = cv2.imread(i,3)
    gray = color.rgb2gray(img)
    x.append(gray)
    y.append(2)
    ind += 1
    if(ind == 20000):
        break

100%|██████████| 520/520 [00:00<00:00, 5373.99it/s]


In [2]:
import EZML as ez
import numpy as np
xt = np.array(x)
y = ez.numerical_to_categorical(y)
yt = np.array(y)
print(xt.shape)
print(yt.shape)

Using TensorFlow backend.


(1303, 48, 48)
(1303, 3)


C:\Users\Abdelrahman Ezz\anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Abdelrahman Ezz\anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Abdelrahman Ezz\anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Abdelrahman Ezz\anaconda3\lib\site-packages\te

In [3]:
#x1,x2,y1,y2 = ez.split_train_test(x,y)

In [7]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, Conv2D,MaxPooling2D,BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

In [8]:
import random
c = list(zip(xt, yt))

random.shuffle(c)
xt, yt = zip(*c)

In [10]:
BATCH_SIZE = 128
import tensorflow
from tensorflow.keras import layers, models

def turn_into_1D(x):
    newX = []
    for i in x:
        newX.append(i.ravel())
    return newX
xt = turn_into_1D(xt)

def create_model2(x=None):
    model = Sequential()
    model.add(layers.Flatten(input_shape=(48*48,)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(16, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    return model
model = create_model2()

In [11]:
model.compile(Adam(lr=0.001),loss="categorical_crossentropy",metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 2304)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                147520    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 16)                1040      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 51        
Total params: 152,771
Trainable params: 152,771
Non-trainable params: 0
_________________________________________________________________


In [12]:
import numpy as np
xt = np.array(xt)
yt = np.array(yt)
xt = xt/255

In [13]:
history = model.fit(xt,yt,batch_size=128,epochs=50)

Epoch 1/50
11/11 [==============================] - 0s 2ms/step - loss: 1.0741 - accuracy: 0.3991
Epoch 2/50
11/11 [==============================] - 0s 2ms/step - loss: 1.0518 - accuracy: 0.4137
Epoch 3/50
11/11 [==============================] - 0s 2ms/step - loss: 1.0203 - accuracy: 0.5434
Epoch 4/50
11/11 [==============================] - 0s 2ms/step - loss: 0.9918 - accuracy: 0.5288
Epoch 5/50
11/11 [==============================] - 0s 2ms/step - loss: 0.9350 - accuracy: 0.6324
Epoch 6/50
11/11 [==============================] - 0s 2ms/step - loss: 0.8445 - accuracy: 0.6585
Epoch 7/50
11/11 [==============================] - 0s 2ms/step - loss: 0.7882 - accuracy: 0.6546
Epoch 8/50
11/11 [==============================] - 0s 2ms/step - loss: 0.7232 - accuracy: 0.7245
Epoch 9/50
11/11 [==============================] - 0s 2ms/step - loss: 0.6330 - accuracy: 0.8127
Epoch 10/50
11/11 [==============================] - 0s 2ms/step - loss: 0.5945 - accuracy: 0.8043
Epoch 11/50
11/11 [

In [14]:
xt.shape

(1303, 2304)

In [15]:
48*48

2304

In [17]:
def landmarks_to_np(landmarks, dtype="int"):
    num = landmarks.num_parts
    
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((num, 2), dtype=dtype)
    
    # loop over the 68 facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, num):
        coords[i] = (landmarks.part(i).x, landmarks.part(i).y)
    # return the list of (x, y)-coordinates
    return coords

def get_centers(frame, landmarks):
   
    EYE_LEFT_OUTTER = landmarks[2]
    EYE_LEFT_INNER = landmarks[3]
    EYE_RIGHT_OUTTER = landmarks[0]
    EYE_RIGHT_INNER = landmarks[1]

    x = ((landmarks[0:4]).T)[0]
    y = ((landmarks[0:4]).T)[1]
    A = np.vstack([x, np.ones(len(x))]).T
    k, b = np.linalg.lstsq(A, y, rcond=None)[0]
    
    x_left = (EYE_LEFT_OUTTER[0]+EYE_LEFT_INNER[0])/2
    x_right = (EYE_RIGHT_OUTTER[0]+EYE_RIGHT_INNER[0])/2
    LEFT_EYE_CENTER =  np.array([np.int32(x_left), np.int32(x_left*k+b)])
    RIGHT_EYE_CENTER =  np.array([np.int32(x_right), np.int32(x_right*k+b)])
    
    pts = np.vstack((LEFT_EYE_CENTER,RIGHT_EYE_CENTER))
    cv2.polylines(frame, [pts], False, (255,0,0), 1) #画回归线
    cv2.circle(frame, (LEFT_EYE_CENTER[0],LEFT_EYE_CENTER[1]), 3, (0, 0, 255), -1)
    cv2.circle(frame, (RIGHT_EYE_CENTER[0],RIGHT_EYE_CENTER[1]), 3, (0, 0, 255), -1)
    
    return LEFT_EYE_CENTER, RIGHT_EYE_CENTER

def get_aligned_face(frame, left, right):
    desired_w = 256
    desired_h = 256
    desired_dist = desired_w * 0.5
    
    eyescenter = ((left[0]+right[0])*0.5 , (left[1]+right[1])*0.5)# 眉心
    dx = right[0] - left[0]
    dy = right[1] - left[1]
    dist = np.sqrt(dx*dx + dy*dy)# 瞳距
    scale = desired_dist / dist # 缩放比例
    angle = np.degrees(np.arctan2(dy,dx)) # 旋转角度
    M = cv2.getRotationMatrix2D(eyescenter,angle,scale)# 计算旋转矩阵

    # update the translation component of the matrix
    tX = desired_w * 0.5
    tY = desired_h * 0.5
    M[0, 2] += (tX - eyescenter[0])
    M[1, 2] += (tY - eyescenter[1])

    aligned_face = cv2.warpAffine(frame,M,(desired_w,desired_h))
    
    return aligned_face

def judge_eyeglass(frame):
    frame = cv2.GaussianBlur(frame, (11,11), 0) #高斯模糊

    sobel_y = cv2.Sobel(frame, cv2.CV_64F, 0 ,1 , ksize=-1) #y方向sobel边缘检测
    sobel_y = cv2.convertScaleAbs(sobel_y) #转换回uint8类型
    cv2.imshow('sobel_y',sobel_y)

    edgeness = sobel_y #边缘强度矩阵
    
    #Otsu二值化
    retVal,thresh = cv2.threshold(edgeness,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    
    #计算特征长度
    d = len(thresh) * 0.5
    x = np.int32(d * 6/7)
    y = np.int32(d * 3/4)
    w = np.int32(d * 2/7)
    h = np.int32(d * 2/4)

    x_2_1 = np.int32(d * 1/4)
    x_2_2 = np.int32(d * 5/4)
    w_2 = np.int32(d * 1/2)
    y_2 = np.int32(d * 8/7)
    h_2 = np.int32(d * 1/2)
    
    roi_1 = thresh[y:y+h, x:x+w] #提取ROI
    roi_2_1 = thresh[y_2:y_2+h_2, x_2_1:x_2_1+w_2]
    roi_2_2 = thresh[y_2:y_2+h_2, x_2_2:x_2_2+w_2]
    roi_2 = np.hstack([roi_2_1,roi_2_2])
    
    measure_1 = sum(sum(roi_1/255)) / (np.shape(roi_1)[0] * np.shape(roi_1)[1])#计算评价值
    measure_2 = sum(sum(roi_2/255)) / (np.shape(roi_2)[0] * np.shape(roi_2)[1])#计算评价值
    measure = measure_1*0.3 + measure_2*0.7
    
    cv2.imshow('roi_1',roi_1)
    cv2.imshow('roi_2',roi_2)
    print(measure)
    
    #根据评价值和阈值的关系确定判别值
    if measure > 0.15:#阈值可调，经测试在0.15左右
        judge = True
    else:
        judge = False
    print(judge)
    return judge

In [23]:
import cv2
import numpy as np
import dlib
thres = 0.5 # Threshold to detect object
nms_threshold = 0.2


classNames= []
classFile = r'C:\Users\Abdelrahman Ezz\Desktop\New folder\Object_Detection_Files\coco.names'
with open(classFile,'rt') as f:
    classNames = f.read().rstrip('\n').split('\n')

#print(classNames)
configPath = r'C:\Users\Abdelrahman Ezz\Desktop\New folder\Object_Detection_Files\ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
weightsPath = r'C:\Users\Abdelrahman Ezz\Desktop\New folder\Object_Detection_Files\frozen_inference_graph.pb'

net = cv2.dnn_DetectionModel(weightsPath,configPath)
net.setInputSize(320,320)
net.setInputScale(1.0/ 127.5)
net.setInputMean((127.5, 127.5, 127.5))
net.setInputSwapRB(True)



classes = ['happy','neutral','sad']
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(r"E:\Z-Machine Learning\My Work\FaceDetection\shape_predictor_68_face_landmarks.dat")
face_cascade = cv2.CascadeClassifier('E:\Z-Machine Learning\My Work\FaceDetection\haarcascade_frontalface_default.xml')
predictor_2_glass =dlib.shape_predictor(r"E:\Z-Machine Learning\My Work\FaceDetection\shape_predictor_5_face_landmarks.dat") 
cap = cv2.VideoCapture(0)
ind = 0
while True:
    success, frame = cap.read()
    
    gray = cv2.cvtColor(src=frame, code=cv2.COLOR_BGR2GRAY)
    frameOrig = frame.copy()
    faces_det = face_cascade.detectMultiScale(gray, 1.3, 5)
    #ind += 1
    
    
    classIds, confs, bbox = net.detect(frame,confThreshold=thres)
    bbox = list(bbox)
    confs = list(np.array(confs).reshape(1,-1)[0])
    confs = list(map(float,confs))
    #print(type(confs[0]))
    #print(confs)

    indices = cv2.dnn.NMSBoxes(bbox,confs,thres,nms_threshold)
    
    for (x,y,w,h) in faces_det:
        expression = gray[y:y+h, x:x+w]
        expression = cv2.resize(expression,(48,48),interpolation=cv2.INTER_CUBIC)
        #cv2.imwrite(r'E:\Z-Machine Learning\My Work\DataSets\Face Expression Recognition\My Own Faces Data\Sad\ ' + str(ind)+'.jpg', expression )
        expression = turn_into_1D(expression)
        expression = np.array(expression)
        expression = expression/255
        test = []
        test.append(expression)
        test = np.array(test)
        pred = model.predict(test)
        pred = ez.categorical_to_numerical(pred)
        position = (x,y-50)
        cv2.putText(
            frame, #numpy array on which text is written
            ("This Person is " + str(classes[pred[0]]) ), position, cv2.FONT_HERSHEY_SIMPLEX, 0.65, (209, 80, 0, 255), #font color
            3) #font stroke
        cv2.imshow('trial', expression)
        
        #ind += 1
    
    
   
    for i in indices:
        i = i[0]
        box = bbox[i]
        x,y,w,h = box[0],box[1],box[2],box[3]
        cv2.rectangle(frame, (x,y),(x+w,h+y), color=(0, 255, 0), thickness=2)
        cv2.putText(frame,classNames[classIds[i][0]-1].upper(),(box[0]+10,box[1]+30),
                    cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
        # cv2.putText(img,str(round(confidence*100,2)),(box[0]+200,box[1]+30),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)

    
    
    
    
    
    faces = detector(gray)
    for face in faces:
        x1_face = face.left()
        y1_face = face.top()
        x2_face = face.right()
        y2_face = face.bottom()
        cv2.rectangle(frame,(x1_face,y1_face),(x2_face,y2_face),(255,0,0),2)
        
        
        landmarks = predictor_2_glass(gray, face)
        landmarks = landmarks_to_np(landmarks)
        for (x, y) in landmarks:
            cv2.circle(frame, (x, y), 2, (0, 0, 255), -1)
        LEFT_EYE_CENTER, RIGHT_EYE_CENTER = get_centers(frame, landmarks)
        aligned_face = get_aligned_face(gray, LEFT_EYE_CENTER, RIGHT_EYE_CENTER)
        judge = judge_eyeglass(aligned_face)
        if judge == True:
            cv2.putText(frame, "With Glasses", (x1_face + 100, y1_face - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)
        else:
            cv2.putText(frame, "No Glasses", (x1_face + 100, y1_face - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2, cv2.LINE_AA)
        
        
        landmarks = predictor(image=gray, box=face)
        for n in range(0, 68):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            
            if(n > 60 and n != 64 and n < 68 ):
                cv2.circle(img=frame, center=(x, y), radius=3, color=(0, 255, 0), thickness=-1)
            else:
                cv2.circle(img=frame, center=(x, y), radius=3, color=(255, 255, 255), thickness=-1)
                
        value = (landmarks.part(8).y + landmarks.part(30).y) /landmarks.part(30).y
        if(landmarks.part(65).y > landmarks.part(63).y + value and landmarks.part(66).y > landmarks.part(62).y + value
            and landmarks.part(67).y > landmarks.part(61).y + value):
            position = (10,50)
            cv2.putText(
                frame, #numpy array on which text is written
                ("A Person is Speaking " ), position, cv2.FONT_HERSHEY_SIMPLEX, 0.65, (209, 80, 0, 255), #font color
                3) #font stroke


        # Read Image
        size = frame.shape

        #2D image points. If you change the image, you need to change vector
        image_points = np.array([
                                    (landmarks.part(30).x, landmarks.part(30).y),     # Nose tip
                                    (landmarks.part(8).x, landmarks.part(8).y),       # Chin
                                    (landmarks.part(36).x, landmarks.part(36).y),     # Left eye left corner
                                    (landmarks.part(45).x, landmarks.part(45).y),     # Right eye right corne
                                    (landmarks.part(48).x, landmarks.part(48).y),     # Left Mouth corner #done
                                    (landmarks.part(54).x, landmarks.part(54).y)      # Right mouth corner #done
                                ], dtype="double")

        # 3D model points.
        model_points = np.array([
                                    (0.0, 0.0, 0.0),             # Nose tip
                                    (0.0, -330.0, -65.0),        # Chin
                                    (-225.0, 170.0, -135.0),     # Left eye left corner
                                    (225.0, 170.0, -135.0),      # Right eye right corne
                                    (-150.0, -150.0, -125.0),    # Left Mouth corner
                                    (150.0, -150.0, -125.0)      # Right mouth corner

                                ])


        # Camera internals

        focal_length = size[1]
        center = (size[1]/2, size[0]/2)
        camera_matrix = np.array(
                                 [[focal_length, 0, center[0]],
                                 [0, focal_length, center[1]],
                                 [0, 0, 1]], dtype = "double"
                                 )

        print ("Camera Matrix :\n {0}".format(camera_matrix))

        dist_coeffs = np.zeros((4,1)) # Assuming no lens distortion
        (success, rotation_vector, translation_vector) = cv2.solvePnP(model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.cv2.SOLVEPNP_ITERATIVE)

        print ("Rotation Vector:\n {0}".format(rotation_vector))
        print ("Translation Vector:\n {0}".format(translation_vector))


        (nose_end_point2D, jacobian) = cv2.projectPoints(np.array([(0.0, 0.0, 1000.0)]), rotation_vector, translation_vector, camera_matrix, dist_coeffs)

        for p in image_points:
            cv2.circle(frame, (int(p[0]), int(p[1])), 3, (0,0,255), -1)


        p1 = ( int(image_points[0][0]), int(image_points[0][1]))
        p2 = ( int(nose_end_point2D[0][0][0]), int(nose_end_point2D[0][0][1]))

        cv2.line(frame, p1, p2, (255,0,0), 2)

        

    
    position = (300,50)
    cv2.putText(
            frame, #numpy array on which text is written
            ("Number of faces "+ str(len(faces)) ), position, cv2.FONT_HERSHEY_SIMPLEX, 0.65, (209, 80, 0, 255), #font color
            2) #font stroke
         
            
    cv2.imshow(winname="Face", mat=frame)
    cv2.imshow(winname='orig',mat=frameOrig)
    if cv2.waitKey(delay=1) == 27:
        break
cap.release()
cv2.destroyAllWindows()

0.07634277343750001
False
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.17423838]
 [-0.05058688]
 [-0.36309364]]
Translation Vector:
 [[ 162.66158567]
 [ -98.1048199 ]
 [3023.10226287]]
0.12329915364583333
False
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.18346207]
 [-0.0319177 ]
 [-0.30452815]]
Translation Vector:
 [[ 159.20862793]
 [ -93.99192977]
 [3042.19077076]]
0.10058186848958334
False
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.1780391 ]
 [-0.01797087]
 [-0.39275132]]
Translation Vector:
 [[ 163.92146713]
 [ -97.03426803]
 [3055.09501531]]
0.07433675130208332
False
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.17006039]
 [-0.046696  ]
 [-0.28222119]]
Translation Vector:
 [[ 156.46473834]
 [ -89.80000734]
 [3033.54960486]]
0.07993570963541666
False
Camera Matrix :
 [[640.   0. 320.]

0.04065348307291666
False
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.14088433]
 [-0.00515447]
 [-0.40281399]]
Translation Vector:
 [[ 182.24521047]
 [ -86.40751215]
 [3011.62517968]]
0.0422607421875
False
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.16122798]
 [-0.02270134]
 [-0.47721463]]
Translation Vector:
 [[ 184.68464738]
 [ -90.64573066]
 [2984.68059575]]
0.03797607421875
False
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.19937239]
 [-0.04170177]
 [-0.30885557]]
Translation Vector:
 [[ 173.64659254]
 [ -94.6215667 ]
 [3029.57383344]]
0.03304036458333333
False
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.17786342]
 [-0.0525849 ]
 [-0.41891365]]
Translation Vector:
 [[ 175.77296769]
 [ -95.01977049]
 [3024.06186632]]
0.05931396484375
False
Camera Matrix :
 [[640.   0. 320.]
 [  0. 64

0.11234537760416666
False
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.20638264]
 [-0.08504526]
 [-0.2801552 ]]
Translation Vector:
 [[ 171.54131714]
 [ -87.74785658]
 [3045.39305753]]
0.008020019531249999
False
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.18328533]
 [-0.09245643]
 [-0.40714319]]
Translation Vector:
 [[ 183.0195804 ]
 [ -83.19565217]
 [3046.48041615]]
0.0131591796875
False
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.18865553]
 [-0.0872701 ]
 [-0.38759294]]
Translation Vector:
 [[ 182.77758844]
 [ -82.67578298]
 [3036.92619274]]
0.10093994140624998
False
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.1792529 ]
 [-0.08881707]
 [-0.33556229]]
Translation Vector:
 [[ 179.0287147 ]
 [ -85.14469666]
 [3020.2838571 ]]
0.006583658854166666
False
Camera Matrix :
 [[640.   0. 320.]
 

0.016589355468749997
False
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.19372367]
 [-0.07001189]
 [-0.23800442]]
Translation Vector:
 [[ 171.65635428]
 [ -82.26528701]
 [3057.81674737]]
0.127734375
False
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.23377845]
 [-0.06588808]
 [ 0.02789136]]
Translation Vector:
 [[ 145.96759435]
 [ -85.09434911]
 [3066.52801647]]
0.14373779296875
False
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.21075058]
 [-0.0574343 ]
 [ 0.13017971]]
Translation Vector:
 [[ 130.98293733]
 [ -86.57452278]
 [3086.37512594]]
0.08132731119791667
False
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.23935067]
 [-0.08011121]
 [ 0.12226142]]
Translation Vector:
 [[ 131.32485189]
 [ -95.46537922]
 [3100.85870018]]
0.10882975260416666
False
Camera Matrix :
 [[640.   0. 320.]
 [  0. 64

0.09774169921875
False
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.03054641]
 [ 0.01032086]
 [ 0.86122685]]
Translation Vector:
 [[  50.87413147]
 [ -52.91134349]
 [3083.09810985]]
0.11512858072916665
False
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.03688261]
 [ 0.0135575 ]
 [ 0.83159977]]
Translation Vector:
 [[  54.41241595]
 [ -55.1410747 ]
 [3068.66842135]]
0.14311116536458332
False
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.01972524]
 [ 0.01821348]
 [ 0.8082173 ]]
Translation Vector:
 [[  57.49224997]
 [ -48.65473901]
 [3090.97984058]]
0.11765950520833332
False
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.05253206]
 [ 0.02658925]
 [ 0.79094564]]
Translation Vector:
 [[  59.56973985]
 [ -57.1694059 ]
 [3107.70525245]]
0.11377360026041666
False
Camera Matrix :
 [[640.   0. 320.]
 [

...

0.04362386067708333
False
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.17761586]
 [-0.05662189]
 [-0.2981575 ]]
Translation Vector:
 [[ 364.00412749]
 [-100.38026795]
 [3030.06083505]]
0.04031168619791667
False
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.19528119]
 [-0.0777891 ]
 [-0.25517633]]
Translation Vector:
 [[ 374.14234934]
 [-103.22307287]
 [3017.07581368]]
0.025284830729166666
False
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.19565962]
 [-0.08682424]
 [-0.29470882]]
Translation Vector:
 [[ 410.16872729]
 [-110.05374021]
 [3071.44781015]]
0.033858235677083334
False
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.20200627]
 [-0.09822034]
 [-0.33935125]]
Translation Vector:
 [[ 435.93866898]
 [-117.69067099]
 [3121.66634513]]
0.02580159505208333
False
Camera Matrix :
 [[640.   0. 320

0.1158447265625
False
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.13805184]
 [ 0.18580362]
 [ 0.60904061]]
Translation Vector:
 [[  68.75707783]
 [-123.78614876]
 [3055.86004494]]
0.07856445312499999
False
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.13946334]
 [ 0.1875593 ]
 [ 0.64136188]]
Translation Vector:
 [[  71.21922331]
 [-142.69506269]
 [3220.48394417]]
0.0786376953125
False
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.14588526]
 [ 0.23185245]
 [ 0.68826596]]
Translation Vector:
 [[  90.48212054]
 [-160.59097489]
 [3298.54278665]]
0.0814208984375
False
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.10887381]
 [ 0.19564425]
 [ 0.81973426]]
Translation Vector:
 [[ 112.74762655]
 [-163.16007328]
 [3243.1798099 ]]
0.12606201171875
False
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 24

0.16793619791666664
True
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-2.95490214]
 [-0.01544063]
 [ 0.99675671]]
Translation Vector:
 [[ 376.55885581]
 [ -42.7896979 ]
 [2967.8346867 ]]
0.14053141276041667
False
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-2.95546748]
 [ 0.00409317]
 [ 1.02086661]]
Translation Vector:
 [[ 375.59769618]
 [ -42.44862721]
 [2968.15981765]]
0.05933024088541666
False
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-2.96289635]
 [ 0.0054183 ]
 [ 1.02306584]]
Translation Vector:
 [[ 378.22937931]
 [ -38.41963627]
 [2969.57015898]]
0.1561279296875
True
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-2.92870239]
 [-0.02238355]
 [ 0.96961129]]
Translation Vector:
 [[ 378.85878715]
 [ -44.28674799]
 [3015.98637187]]
0.11241455078125
False
Camera Matrix :
 [[640.   0. 320.]
 [  0. 6

0.09518636067708333
False
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.02955703]
 [ 0.18597272]
 [ 0.78491328]]
Translation Vector:
 [[ 291.05970029]
 [ -78.90103906]
 [3030.53775074]]
0.113818359375
False
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-2.92070986]
 [ 0.22022628]
 [ 1.03647497]]
Translation Vector:
 [[ 244.5964286 ]
 [ -82.59589975]
 [3071.61877174]]
0.200927734375
True
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.02437692]
 [ 0.21975382]
 [ 0.79655075]]
Translation Vector:
 [[ 218.29678828]
 [-113.44788047]
 [3165.43766181]]
0.24571940104166667
True
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.01920215]
 [ 0.24790919]
 [ 0.75402783]]
Translation Vector:
 [[ 201.69032116]
 [-120.55933272]
 [3181.62954639]]
0.21960449218749997
True
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 

0.10607910156249999
False
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-2.99856371]
 [ 0.14839288]
 [ 0.8841591 ]]
Translation Vector:
 [[ -56.43959308]
 [ -95.20854224]
 [3044.58401675]]
0.15693359375
True
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.03478995]
 [ 0.10703092]
 [ 0.92122342]]
Translation Vector:
 [[ -48.88465672]
 [-130.35654851]
 [2862.413056  ]]
0.16415608723958333
True
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-2.99733258]
 [ 0.07315599]
 [ 0.95031673]]
Translation Vector:
 [[ -46.04934173]
 [-119.76960538]
 [2928.61802184]]
0.05899251302083333
False
Camera Matrix :
 [[640.   0. 320.]
 [  0. 640. 240.]
 [  0.   0.   1.]]
Rotation Vector:
 [[-3.12708521]
 [ 0.11135606]
 [ 0.77966856]]
Translation Vector:
 [[  23.96134694]
 [-145.50604858]
 [3121.19189442]]
0.07616780598958332
False
Camera Matrix :
 [[640.   0. 320.]
 [  0. 